In [1]:
import os
# os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
import importlib
import common_utils
importlib.reload(common_utils)
import pandas as pd
import itertools

import tensorflow as tf
from common_utils import get_unique_image_shapes,get_unique_image_paths,load_images_from_paths,build_image_dataframe,split_data, bin_ages, build_cnn_model,build_model_from_config

In [2]:


# tf.keras.mixed_precision.set_global_policy('mixed_float16')

# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)


In [3]:
image_paths_csv = pd.read_csv('./processed_data/image_paths.csv')

In [4]:
paths_train_df, paths_val_df, paths_test_df = split_data(image_paths_csv)


In [5]:
paths_train_df

,path,age,age_bin,age_bin_label
0,../raw_data/face_age/035/1514.png,35,7,Mid Adults (35–44)
1,../raw_data/face_age/055/3004.png,55,9,Older Adults (55–64)
2,../raw_data/face_age/015/9256.png,15,4,Teens (13–17)
3,../raw_data/face_age/025/7567.png,25,6,Adults (25–34)
4,../raw_data/face_age/009/7776.png,9,3,Pre-teens (9–12)
...,...,...,...,...
6887,../raw_data/face_age/037/2544.png,37,7,Mid Adults (35–44)
6888,../raw_data/face_age/085/6833.png,85,12,Oldest (85+)
6889,../raw_data/face_age/030/5693.png,30,6,Adults (25–34)
6890,../raw_data/face_age/032/9532.png,32,6,Adults (25–34)


In [6]:
# Converting the filenames and target class labels into lists for augmented train and test datasets.

test_filenames_list = list(paths_train_df['path'])
test_labels_list = list(paths_train_df['age_bin'])

test_filenames_tensor = tf.constant(test_filenames_list)
test_labels_tensor = tf.constant(test_labels_list)

val_filenames_list = list(paths_val_df['path'])
val_labels_list = list(paths_val_df['age_bin'])

val_filenames_tensor = tf.constant(val_filenames_list)
val_labels_tensor = tf.constant(val_labels_list)


2025-04-01 17:27:47.530259: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-04-01 17:27:47.530285: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-01 17:27:47.530289: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-01 17:27:47.530304: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-01 17:27:47.530317: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
batch_size = 128
configurations = {
    'channels': [3],
    'num_conv_layers': [3,4,5],
    'base_filters': [32,64,128],
    'kernel_size': [3,5],
    'activation': ['relu','tanh','swish'],
    'use_skip': [False,True],
    'num_dense_layers': [1,2,3],
    'dense_units': [32,64,128,256],
    'num_classes': [13],
    'dropout_rate': [0.3, 0.5,0.7],
    'output_activation': ['softmax'],
    'pool_size': [2,3],
    'task': ['classification']
}


In [8]:


all_combinations = list(itertools.product(*configurations.values()))
valid_configs = []

for combo in all_combinations:
    params = dict(zip(configurations.keys(), combo))
    
    # Enforce: If use_skip=True, use_pooling=False
    if params['use_skip']:
        params['use_pooling'] = False
    else:
        params['use_pooling'] = True  # Or add to search space
    
    valid_configs.append(params)



In [9]:
train_dataset = load_images_from_paths(test_filenames_tensor,test_labels_tensor, channels=3,ratio=1, batch_size=batch_size)
val_dataset = load_images_from_paths(val_filenames_tensor,val_labels_tensor, channels=3,ratio=1,batch_size=batch_size)


In [10]:
# def configure_dataset(dataset, shuffle=False):
#     # if shuffle:
#     #     dataset = dataset.shuffle(buffer_size=500)  # Adjust buffer size as needed
#     return dataset


In [1]:
# Creating a TensorBoard callback object and saving it at the desired location.
# from tensorflow.python.keras.callbacks import ModelCheckpoint
# # tensorboard = TensorBoard(log_dir=f"./tensorboard/1.1_age_input_output/output/cnn_logs")
# 
# # Creating a ModelCheckpoint callback object to save the model according to the value of val_accuracy.
# 
# checkpoint = ModelCheckpoint(filepath=f"/content/drive/My Drive/1_LiveProjects/Project5_AgeGenderEmotion_Detection/1.1_age_input_output/output/cnn_logs/age_model_checkpoint.h5",
#                              monitor='val_accuracy',
#                              save_best_only=True,
#                              save_weights_only=False,
#                              verbose=1
#                             )

NameError: name 'TensorBoard' is not defined

In [ ]:
model_dict = {}
model_dir = "saved_models"
os.makedirs(model_dir, exist_ok=True)
paths_pre_train_model_dict = {}


for i, config in enumerate(valid_configs):
    # Generate unique model name
    name_parts = [
        f"{config['channels']}ch",
        f"skip_{config['use_skip']}",
        f"conv{config['num_conv_layers']}",
        f"k{config['kernel_size']}",
        config['activation'],
        f"dense{config['num_dense_layers']}x{config['dense_units']}",
        f"drop{config['dropout_rate']}",
        f"out_{config['output_activation']}"
    ]
    
    model_name = "_".join(name_parts)
    # model_filename = model_name + ".keras"
    # model_path = os.path.join(model_dir, model_filename)

    # Build and compile
    model = build_model_from_config(config)
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',  # Fixed loss function
        metrics=['accuracy']
    )
    # model.save(model_path,overwrite=True)
    # model.summary()
    # paths_pre_train_model_dict[model_name] = {
    #     'model_path': model_path,
    #     'config': config
    # }
    # 
    # # Train
    history = model.fit(
        train_dataset,  # Dataset yields (images, targets)
        validation_data=val_dataset,
        epochs=50,
        batch_size=batch_size,
        verbose=2,
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)]
    )

    # Store results
    model_dict[model_name] = {
        'model': model,
        'history': history.history,
        'config': config
    }
    print(f"Trained {model_name} | Val acc: {max(history.history['val_accuracy']):.4f}")
    # print(f"Created {model_name} ")


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cast (Cast)                     │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 200, 200, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 200, 200, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 100, 100, 64)   │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 100, 100, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 100, 100, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 50, 50, 128)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 50, 50, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 50, 50, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_2             │ (None, 25, 25, 256)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 132)            │        33,924 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 13)             │         1,729 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 408,261 (1.56 MB)

 Trainable params: 407,365 (1.55 MB)

 Non-trainable params: 896 (3.50 KB)

Epoch 1/50


2025-04-01 17:27:48.351644: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


108/108 - 117s - 1s/step - accuracy: 0.2073 - loss: 2.4318 - val_accuracy: 0.1706 - val_loss: 2.6611
Epoch 2/50


In [ ]:
model_dict['1ch_skip_False_conv3_k3_relu_dense1x132_out_softmax']['history']['accuracy']